# Subqueries

A subquery, simply put, is a query written as a part of a bigger statement.
Think of it as a SELECT statement inside another one.

The result of the
inner SELECT can then be used in the outer query.

```
tesdb=# SELECT author FROM authors_tbl
        WHERE language_id IN
            ( 
                SELECT id FROM newlang_tbl
                WHERE language='Tcl'
            );
```

The subquery SELECT id FROM newlang_tbl WHERE language='Tcl'
picks the correct language id from the newlang_tbl and passes it on to the
outer query on the authors table. 

This frees us from the responsibility of
joining the two tables using the language id field.

We can visualize the intermediate step where the subquery has already
resolved to a value. The query would now look something like ``SELECT
author FROM authors_tbl WHERE language_id IN (4)``.


Remember that all
joins can be rewritten as subqueries, but the reverse is not true in all cases.


# Types of Subqueries
 
* __Scalar subqueries__ A subquery that returns only
a single column of a single row as its output.
* __Row subqueries__ A subquery that returns a single
row but more than one column.
* __Table subqueries__ return more
than a single row and many columns per row.  It can return a table itself to take part in
your outer query.


A table subquery example

```
tesdb=# SELECT author, language
        FROM authors_tbl a,
            (
                SELECT id, language
                FROM newlang_tbl
                WHERE year > 1980
            ) new
        WHERE a.language_id = new.id;
```

# Existence Tests in Subqueries

The keyword EXISTS tests the presence of any number of rows returned
from a subquery.

```
tesdb=# SELECT language,
                year
        FROM newlang_tbl
        WHERE EXISTS (
            SELECT * FROM authors_tbl
            WHERE newlang_tbl.id = language_id
            );
```

=> Alle talen die ook een ook in de language_id kolom bestaan van authors_tbl.


Notice the subquery WHERE clause in this case. It is effectively referencing
the outer table field using newlang_tbl.id.

We can add the option NOT to the existence test to find the complement
of the result 


```
tesdb=# SELECT language,
                year
        FROM newlang_tbl
        WHERE NOT EXISTS (
            SELECT * FROM authors_tbl
            WHERE newlang_tbl.id = language_id
            );
```

Recall that we had never put the corresponding entry in the authors
table for Lisp in the last chapter.

# Using Subqueries in INSERT Statements

We can even use subqueries inside other SQL statement like INSERT. 

Let us try to add a new language and a new author in our tables and ease our task of remembering id numbers by just a bit by using subqueries

```
tesdb=# INSERT INTO newlang_tbl
        (id, language, year, standard)
        VALUES (7, 'Pascal', 1970, 'ISO');

tesdb=# SELECT * FROM newlang_tbl;

tesdb=# SELECT * FROM authors_tbl;  
```

While inserting a new entry into the authors_tbl, we can either
remember that we used the language_id as 7 for Pascal or use a subquery.

```
tesdb=# INSERT INTO authors_tbl
        (author_id, author, language_id)
        VALUES (7, 'Wirth',
        (
            SELECT id FROM newlang_tbl WHERE language='Pascal')
        );

tesdb=# SELECT * FROM authors_tbl;        
```

You can even use subqueries to control your UPDATE and DELETE
statements. The logic remains much the same as with using subqueries in
SELECT and INSERT.

# Using ANY and ALL

The ANY operator used with the arithmetic comparison operators can
be used to check a column value in comparison to a similar value(s)
generated in the subquery. 

For example, if we wanted to display all the
languages but exclude the oldest one from the result, we could combine >
and ANY to achieve this

```
tesdb=# SELECT language
        FROM newlang_tbl
        WHERE year > ANY (SELECT year FROM newlang_tbl);
```

Only Lisp does not have a creation year that is not greater than any of
the list of values returned by the subquery.

what would happen if we reversed our comparison operator to
< ANY? 

```
tesdb=# SELECT language
        FROM newlang_tbl
        WHERE year < ANY (SELECT year FROM newlang_tbl);
```

Tcl is notably
absent. This is because the year of Tcl, that is, 1988 is not less than any of
the values returned by the subquery. 



The other comparison conjunction we can use with ANY is =, but that
is rarely seen because it is equivalent to using IN (), 

The ALL operator works similarly, but the value in the WHERE clause
must hold true for all of the values returned from the subquery 


```
tesdb=# SELECT language
        FROM newlang_tbl
        WHERE year <= ALL (SELECT year FROM newlang_tbl);

tesdb=# SELECT language
        FROM newlang_tbl
        WHERE year >= ALL (SELECT year FROM newlang_tbl);    
```